In [16]:
#Импортируем необходиимых библиотек для реализации парсинга
from selenium import webdriver
from selenium_stealth import stealth # делает похожим на настоящего пользователя
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np

In [109]:
options = webdriver.ChromeOptions()
options.add_argument("start_maximized")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
#chromedriver должен соответствовать актуальной версии chrom, тут используется версия chrom 107.0.5304.121
#версия chromedriver - 107.0.5304.62 (все работает!)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

imo = []

for page in tqdm(range(0, 21500, 500), 'Обработанно страниц'):
    url = f"https://fleetphoto.ru/list.php?rgid=2&st={page}" #загружаем русскоязычную версию сайта
    driver.get(url)
    
    if page == 21000:
        for j in range(1, 151):
            try:
                soup = BeautifulSoup(driver.page_source, 'lxml') #передаем данные в суп
                data1 = soup.find('div', class_ = 'rtable').find('table').findAll('tr')[j]
                dates_imo = data1.findAll('td')[4].text #выгружаем наименования
                imo.append(dates_imo)
            except:
                dates_imo = 0
                imo.append(dates_imo)
                continue
    else:
        for j in range(1, 501):
            try:
                if page > 2000: #с 6-го листа появляется новый столбец Рег. № и ИМО становится 4-м
                    soup = BeautifulSoup(driver.page_source, 'lxml') #передаем данные в суп
                    data1 = soup.find('div', class_ = 'rtable').find('table').findAll('tr')[j]
                    dates_imo = data1.findAll('td')[4].text #выгружаем наименования
                    imo.append(dates_imo)
                else: #до 5-го листа включительно ИМО является 3-м
                    soup = BeautifulSoup(driver.page_source, 'lxml') #передаем данные в суп
                    data1 = soup.find('div', class_ = 'rtable').find('table').findAll('tr')[j]
                    dates_imo = data1.findAll('td')[3].text #выгружаем наименования
                    imo.append(dates_imo)
            except:
                dates_imo = 0
                imo.append(dates_imo)
                continue

C:\Users\skapr\AppData\Local\Temp\ipykernel_9792\29343523.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
Обработанно страниц: 100%|██████████████████████████████████████████████████████████| 43/43 [1:55:15<00:00, 160.82s/it]


In [110]:
#создаем DF
df_imo = pd.DataFrame(imo)
df_imo

,0
0,9916795
1,9916800
2,
3,
4,8722238
...,...
21145,9759939
21146,
21147,6703630
21148,6719809


In [111]:
#Сохраняем в Excel
writer = pd.ExcelWriter('Флитфото.xlsx')
df_imo.to_excel(writer, 'imo')
writer.save()